In [57]:
import os
import requests
from datetime import datetime, timedelta
import pandas as pd
from meteostat import Point, Daily
from google.colab import files

***Input*** ***1: Crop Information: Names & Locations***

In [58]:
df = pd.read_excel("harvest_locations.xlsx", skiprows=1)

header_names = df.iloc[1]
state_location = df["Kt."]
origin_location = df["Standort"]
crop_names = df.columns[7:].tolist()

locations_dict = {}

for crop in crop_names:
    locations_dict[crop] = df.loc[df[crop] == "x", "Standort"].tolist()

***Input 2: Climate Data***

* Location

In [59]:
latitude = 47.3769  # Zurich lat
longitude = 8.5417  # Zurich lon

* Historical climate data

In [60]:
def get_historical_weather(start_date_str, end_date_str):
    """
    Fetch historical daily weather (ERA5).
    ERA5 typically has ~5 days delay, so yesterday is not available.
    """
    ERA5_URL = "https://archive-api.open-meteo.com/v1/era5"
    # Adjust end_date if too recent
    today = date.today()
    max_available = today - timedelta(days=5)
    end_date = min(datetime.strptime(end_date_str, "%Y-%m-%d").date(), max_available)
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d").date()

    if start_date > end_date:
        raise ValueError("Start date is after available end date.")

    daily_vars = ",".join([
        "temperature_2m_max","temperature_2m_min","precipitation_sum",
        "windspeed_10m_max","windgusts_10m_max","shortwave_radiation_sum"
    ])

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat(),
        "daily": daily_vars,
        "timezone": "Europe/Zurich"
    }
    resp = requests.get(ERA5_URL, params=params)
    if resp.status_code == 200:
        df = pd.DataFrame(resp.json()["daily"])
        # Simple weathercode
        df["weathercode"] = df.apply(lambda row: "rain" if row["precipitation_sum"]>5 else
                                     "showers/windy" if row["precipitation_sum"]>0 and row["windspeed_10m_max"]>10 else
                                     "sunny" if row["shortwave_radiation_sum"]>15000 else "cloudy", axis=1)
        return df
    else:
        print("Historical request failed:", resp.status_code)
        return None

* Future weather forecast

In [61]:
# -------------------------------
# 2️⃣ Future weather forecast
# -------------------------------
def get_forecast_weather(n_days):
    """
    Fetch forecast daily weather for the next n_days.
    Converts official weathercode to simplified types like historical data.
    """
    FORECAST_URL = "https://api.open-meteo.com/v1/forecast"
    start_date = date.today()
    end_date = start_date + timedelta(days=n_days-1)

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max,weathercode",
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat(),
        "timezone": "Europe/Zurich"
    }
    resp = requests.get(FORECAST_URL, params=params)
    if resp.status_code == 200:
        df = pd.DataFrame(resp.json()["daily"])

        # map official weathercode to simplified types
        def code_to_simple(wc, precip, wind, rad=0):
            if precip > 5:
                return "rain"
            elif precip > 0 and wind > 10:
                return "showers/windy"
            elif wc in [0, 1, 2]:  # clear/sunny codes
                return "sunny"
            else:
                return "cloudy"

        df["weathercode"] = df.apply(lambda row: code_to_simple(row["weathercode"], row["precipitation_sum"], row["windspeed_10m_max"]), axis=1)
        return df
    else:
        print("Forecast request failed:", resp.status_code)
        return None


In [62]:
if __name__ == "__main__":
    print("=== Historical Weather ===")
    hist_df = get_historical_weather("2025-09-15", "2025-09-25")
    print(hist_df)

    print("\n=== Forecast Weather ===")
    fc_df = get_forecast_weather(7)  # next 5 days
    print(fc_df)

=== Historical Weather ===
         time  temperature_2m_max  temperature_2m_min  precipitation_sum  \
0  2025-09-15                24.6                15.2                6.3   
1  2025-09-16                19.0                13.2                1.3   
2  2025-09-17                19.7                10.4                0.0   
3  2025-09-18                24.3                11.7                0.0   
4  2025-09-19                24.0                12.5                0.0   
5  2025-09-20                27.1                13.6                0.0   
6  2025-09-21                22.8                16.2               14.3   
7  2025-09-22                16.6                12.3               22.1   
8  2025-09-23                13.3                10.8                3.5   

   windspeed_10m_max  windgusts_10m_max  shortwave_radiation_sum  \
0               22.4               50.4                    17.63   
1               17.7               41.8                    14.46   
2       